神经网络工具箱 nn

https://github.com/chenyuntc/pytorch-book/blob/master/chapter04-neural_network/chapter4.ipynb

torch.nn是专门为深度学习而设计的模块。torch.nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。下面先来看看如何用nn.Module实现自己的全连接层。全连接层，又名仿射层，输出$\textbf{y}$和输入$\textbf{x}$满足$\textbf{y=Wx+b}$，$\textbf{W}$和$\textbf{b}$是可学习的参数

In [6]:
import torch as t
from torch import nn

In [12]:
class Linear(nn.Module): # 必须继承自 nn.Module
    def __init__(self,in_feature,out_feature): 
        """
        :param in_feature : 维度
        """
        super(Linear, self).__init__()  # 等价于nn.Module.__init__(self)
        print(type(in_feature),type(out_feature))
        self.w = nn.Parameter(t.randn(in_feature,out_feature)) # 注意 randn 不是 random, 生成一个 in*out的随机数矩阵
        self.b = nn.Parameter(t.randn(out_feature))
    
    def forward(self,x):
        x = x.mm(self.w) # wx
        return x + self.b.expand_as(x)

In [13]:
layer = Linear(4,3) # 

<class 'int'> <class 'int'>


In [15]:
layer.w


Parameter containing:
tensor([[ 0.4689,  0.8883,  1.1222],
        [ 0.3957, -0.2459,  0.1367],
        [ 0.1343, -1.1896,  0.4446],
        [ 1.0134, -0.1258,  1.3106]], requires_grad=True)

In [17]:
input = t.randn(2,4) # x 矩阵
output = layer(input) # nn.Module 已经定义了 __call__() 函数,在其中会自动调用 forward 函数
output   # 这里就执行了一次 y = wx+b 的前向传播计算

tensor([[-2.3089, -0.0981, -1.8870],
        [-1.4828, -0.0403, -1.2181]], grad_fn=<AddBackward0>)

自定义网络的小结
- 自定义层Linear必须继承nn.Module，并且在其构造函数中需调用nn.Module的构造函数，即super(Linear, self).__init__() 或nn.Module.__init__(self)，推荐使用第一种用法，尽管第二种写法更直观。
- 在构造函数__init__中必须自己定义可学习的参数，并封装成Parameter，如在本例中我们把w和b封装成parameter。parameter是一种特殊的Tensor，但其默认需要求导（requires_grad = True），感兴趣的读者可以通过nn.Parameter??，查看Parameter类的源代码。
- forward函数实现前向传播过程，其输入可以是一个或多个tensor。
- 无需写反向传播函数，nn.Module能够利用autograd自动实现反向传播，这点比Function简单许多。
- 使用时，直观上可将layer看成数学概念中的函数，调用layer(input)即可得到input对应的结果。它等价于layers.__call__(input)，在__call__函数中，主要调用的是 layer.forward(x)，另外还对钩子做了一些处理。所以在实际使用中应尽量使用layer(x)而不是使用layer.forward(x)，关于钩子技术将在下文讲解。
- Module中的可学习参数可以通过named_parameters()或者parameters()返回迭代器，前者会给每个parameter都附上名字，使其更具有辨识度。

Module能够自动检测到自己的Parameter，并将其作为学习参数。除了parameter之外，Module还包含子Module，主Module能够递归查找子Module中的parameter。下面再来看看稍微复杂一点的网络，多层感知机。

In [ ]:
class Perceptron(nn.Model):
    def __init__(self,in_features, hidden_features, out_features): # 定义各层参数的个数
        self.layer1 = Linear(in_features, hidden_features) # Linear 是前面定义好的
        self.layer2 = Linear(hidden_features,out_features)
    
    def forward(self,x):
        x = self.layer1(x)
        x = 